### Map instantiedata rioleringen Utrecht naar NLCS Classificatie en Tekenstandaard obv IMBOR classificaties

Momenteel is er nog weinig gemapt met IMBOR concepten, en GWSW mappingen ontbreken in deze dataset. Er is een mapping naar imbor:Mantelbuis gemaakt vanuit NLCS Classificatie, en deze zal hier gebruikt worden om de toepassing te bewijzen.

De gebruikte query bij <https://datasets.crow.nl/redmer-kronemeijer/utrecht-riolerling-rev/sparql>:

```
select * 
where {
  ?s ?p ?o .
  ?s a <https://data.crow.nl/imbor/def/ea3de512-7af7-439f-bdd4-1ddacf6cc2bc> .
} #limit 10
```

In [ ]:
import pandas as pd
from rdflib import Graph, URIRef, Literal
from rdflib.namespace import RDF, SKOS

dfMantelbuis = pd.read_csv("resources/utrecht-riolering-rev-imbor.csv")  #, dtype=str)

g = Graph().parse("resources/NLCS_Classificatie_Kern_v0_0_4.ttl")

In [ ]:
nlcsClassQuery = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX imbor: <https://data.crow.nl/imbor/def/>

SELECT ?uri ?nlcsLaag
WHERE {
    ?uri skos:exactMatch imbor:ea3de512-7af7-439f-bdd4-1ddacf6cc2bc .
    #OPTIONAL {
        ?uri rdfs:seeAlso ?nlcsLaag .
        #FILTER (CONTAINS(STR(?nlcsLaag), "http://digitalbuildingdata.tech/nlcs/def/") 
    #}
} #LIMIT 1
"""

results = g.query(nlcsClassQuery)
for res in results:
    uri = res.uri
    if res.nlcsLaag:
        if "http://digitalbuildingdata.tech/nlcs/def/" in str(res.nlcsLaag):
            nlcsLaag = res.nlcsLaag

In [ ]:
for id in dfMantelbuis[dfMantelbuis.p == "http://www.w3.org/1999/02/22-rdf-syntax-ns#type"].s:
    dfMantelbuis.loc[len(dfMantelbuis)] = [id, RDF.type, uri]
    dfMantelbuis.loc[len(dfMantelbuis)] = [id, SKOS.related, nlcsLaag]
    # print("added")

In [ ]:
ng = Graph()
ng.bind("imbor", "https://data.crow.nl/imbor/def/")
ng.bind("geo", "http://www.opengis.net/ont/geosparql#")
ng.bind("nlcs-class", "http://digitalbuildingdata.tech/nlcs-class/def/")
ng.bind("nlcs", "http://digitalbuildingdata.tech/nlcs/def/")
ng.bind("nen2660", "https://w3id.org/nen2660/def#")
ng.bind("imborConversie", "http://www.wistor.nl/imborconversie#")
ng.bind("skos", SKOS)

for _, row in dfMantelbuis.iterrows():
    if "http" in row.o:
        o = URIRef(row.o)
    else:
        o = Literal(row.o)
    ng.add((URIRef(row.s), URIRef(row.p), o))

ng.serialize("resources/utrecht-riolering-rev-imbor-nlcs.ttl")